<a href="https://colab.research.google.com/github/PandoraCheeze/ppw/blob/main/PPW_Crawling_data_lengkap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
!pip install requests
!pip install html5lib
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=87d75926813fea731cb790d0fa8724dffe1bee1285a6205411693445266182f0
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [2]:
# constant
LAST_INDEX = -1
FIRST_INDEX = 1
INCREMENT_BY_ONE = 1

# import library
from bs4 import BeautifulSoup
import requests

def request_url(url):
    return requests.get(url)

def request_header_url(header, url_website):
    headers = {'User-Agent': header}
    return requests.get(url=url_website, headers=headers)

def parse_website(request):
    """Use html5lib library to parse"""
    return BeautifulSoup(request.content, 'html5lib')

def prettify_web_structure(parsed_page):
    parsed_page.prettify()

def get_content_table(web_element, tag, attributes):
    return web_element.find(tag, attrs = attributes)

# max_pages = 172 # total halaman di website PTA, terbaru februari 2023

r = request_header_url("Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1", "https://pta.trunojoyo.ac.id/c_search/byprod/10")
soup = parse_website(r)
prettify_web_structure(soup)
table = get_content_table(soup, "div", {"id":"wrapper"})

pagination = table.findAll("a", attrs = {"class":"pag_button"})
total_pages = int(pagination[LAST_INDEX]["href"].split("/")[LAST_INDEX])

papers = []
for pages in range(FIRST_INDEX, total_pages + INCREMENT_BY_ONE):

    url_link = "https://pta.trunojoyo.ac.id/c_search/byprod/10"
    id_prodi = url_link.split("/")[LAST_INDEX]
    nama_prodi = ""
    if (id_prodi == "10"):
        nama_prodi = "Teknik Informatika"

    r_pages = request_header_url("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", f"{url_link}/{pages}")
    soup_pages = parse_website(r_pages)
    prettify_web_structure(soup_pages)
    table_pages = get_content_table(soup_pages, "div", {"id":"wrapper"})

    for article_row in table_pages.findAll("a", attrs = {"class":"gray button"}):
        r_article_row = request_header_url("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", article_row["href"])
        soup_article_row = parse_website(r_article_row)
        prettify_web_structure(soup_article_row)
        table_article_row = get_content_table(soup_article_row, "li", {"data-id":"id-1", "data-cat":"#luxury"})
        article_row_title = table_article_row.find('a', attrs = {"class":"title", "href":"#"}).string
        article_row_penulis = ""
        article_row_pembimbing1 = ""
        article_row_pembimbing2 = ""
        article_row_abstract = table_article_row.find('p', attrs = {"align":"justify"}).string
        spans = table_article_row.find_all('span')
        for span in spans:
          span_text = span.get_text(strip=True)
          if "Penulis :" in span_text:
            article_row_penulis = span_text.replace("Penulis :", "").strip()
          elif "Dosen Pembimbing I :" in span_text:
            article_row_pembimbing1 = span_text.replace("Dosen Pembimbing I :", "").strip()
          elif "Dosen Pembimbing II :" in span_text:
            article_row_pembimbing2 = span_text.replace("Dosen Pembimbing II :", "").strip()

        papers.append([
                article_row_title,
                article_row_penulis,
                article_row_pembimbing1,
                article_row_pembimbing2,
                article_row_abstract
        ])

In [7]:
# Membuat DataFrame
columns = ['Title', 'Penulis', 'Pembimbing1', 'Pembimbing2', 'Abstract']
df = pd.DataFrame(papers, columns=columns)

# Menampilkan keseluruhan DataFrame tanpa batasan baris
pd.set_option('display.max_rows', None)

# Menampilkan DataFrame sebagai tabel
from IPython.display import display
display(df)

,Title,Penulis,Pembimbing1,Pembimbing2,Abstract
0,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,A.Ubaidillah S.Kom,Budi Setyono M.T,Hermawan S.T,Sistem informasi akademik (SIAKAD) merupaka...
1,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"M. Basith Ardianto,","Drs. Budi Soesilo, MT","Koko Joni, ST",Berjalannya koneksi jaringan komputer dengan l...
2,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...,"Akhmad Suyandi, S.Kom","Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Web server adalah sebuah perangkat lunak serve...
3,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Heri Supriyanto,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Penjadwalan kuliah di Perguruan Tinggi me...
4,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Septian Rahman Hakim,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Seiring perkembangan teknologi yang ada diduni...
5,Gerak Pekerja Pada Game Real Time Strategy Men...,Adi Chandra Laksono,"Kurniawan Eka P, S.Kom., Msc","Arik Kurniawati, S.Kom., M.T.",Gerak pekerja ada pada game yang memiliki genr...
6,RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MEN...,NURRACHMAT,"Arik Kurniawati, S.Kom., M.T.","Kurniawan Eka Permana, S.Kom., MSc.","Perkembangan game yang semakin pesat, memberik..."
7,EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEA...,Muhammad Choirur Rozi,"Dr. Arif Muntasa, S.Si.,M.T","Fitri Damayanti, S.Kom.,M.kom",Sistem pengenalan wajah adalah suatu sistem un...
8,IMPLEMENTASI ALGORITMA PRIM DAN DEPTH FIRST ...,M Khoiril Anwar,"Cucun Very Angkoso, S.T., M.T.","Arik Kurniawati S. Kom., M.T.",Teknologi mobile game beroperating system open...
9,Perancangan Sistem Informasi Badan Kepegawaian...,MALIKUL HAMZAH,"Moch. Kautsar Sophan, S.Kom., M.MT.","Yeni Kustiyaningsih, S.Kom., M.Kom.",Kantor Badan Kepegawaian kota Bangkalan adalah...


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Preprocess the abstracts (assuming the abstracts are in the 'Abstract' column of the DataFrame df)
abstracts = abstracts.fillna('')

# Tokenize and remove stop words
stop_words = []  # You should populate this with appropriate stop words for your case
tokenized_abstracts = abstracts.apply(lambda x: x.lower().split() if x else [])  # Convert to lowercase and tokenize

# Remove stop words
tokenized_abstracts = tokenized_abstracts.apply(lambda x: [word for word in x if word not in stop_words])

# Combine the tokens back into sentences (abstracts)
preprocessed_abstracts = tokenized_abstracts.apply(lambda x: ' '.join(x))


# Step 2: Convert abstracts to a format suitable for LDA (using CountVectorizer)
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_word_matrix = vectorizer.fit_transform(preprocessed_abstracts)

# Step 3: Train the LDA model
k = 3  # Number of topics
alpha = 0.1
beta = 0.2

lda_model = LatentDirichletAllocation(n_components=k, doc_topic_prior=alpha, topic_word_prior=beta, random_state=42)
lda_model.fit(doc_word_matrix)

# Step 4: Extract and display the results
# Proportions of topics for each document
doc_topic_proportions = lda_model.transform(doc_word_matrix)

# Display the proportions of topics for each document
print("Proportions of topics for each document:")
print(doc_topic_proportions)

# Word distributions for each topic
topic_word_distributions = lda_model.components_

# Display the word distributions for each topic
print("\nWord distributions for each topic:")
feature_names = vectorizer.get_feature_names_out()
for i, topic_dist in enumerate(topic_word_distributions):
    top_words = [feature_names[idx] for idx in topic_dist.argsort()[:-10:-1]]
    print(f"Topic {i+1}: {', '.join(top_words)}")


Proportions of topics for each document:
[[1.75915586e-01 8.23376523e-01 7.07891090e-04]
 [1.12310366e-01 2.89628269e-01 5.98061364e-01]
 [6.20082039e-04 6.20085059e-04 9.98759833e-01]
 ...
 [9.99203990e-01 3.98000641e-04 3.98008915e-04]
 [9.99053288e-01 4.73337146e-04 4.73375091e-04]
 [7.06689514e-01 5.14668546e-02 2.41843632e-01]]

Word distributions for each topic:
Topic 1: dan, dengan, untuk, pada, metode, dari, menggunakan, ini, citra
Topic 2: dan, dengan, sistem, dalam, untuk, ini, dari, data, metode
Topic 3: dan, dengan, ini, pada, untuk, dalam, dari, menggunakan, dapat
